### Import Libraries

In [1]:
import pyforest

### Read Datasets

In [2]:
category_df = pd.read_csv('TMP/category_tree.csv')
category_df.head()

<IPython.core.display.Javascript object>

,categoryid,parentid
0,1016,213.0
1,809,169.0
2,570,9.0
3,1691,885.0
4,536,1691.0


In [3]:
category_df.shape

(1669, 2)

In [4]:
category_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1669 entries, 0 to 1668
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   categoryid  1669 non-null   int64  
 1   parentid    1644 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 26.2 KB


In [5]:
category_df.isnull().sum()

categoryid     0
parentid      25
dtype: int64

In [6]:
events_df = pd.read_csv('TMP/events.csv')
events_df.head()

<IPython.core.display.Javascript object>

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN


In [7]:
events_df.shape

(2756101, 5)

In [8]:
events_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2756101 entries, 0 to 2756100
Data columns (total 5 columns):
 #   Column         Dtype  
---  ------         -----  
 0   timestamp      int64  
 1   visitorid      int64  
 2   event          object 
 3   itemid         int64  
 4   transactionid  float64
dtypes: float64(1), int64(3), object(1)
memory usage: 105.1+ MB


In [9]:
events_df.isnull().sum()

timestamp              0
visitorid              0
event                  0
itemid                 0
transactionid    2733644
dtype: int64

In [10]:
item_properties_1_df = pd.read_csv('TMP/item_properties_part1.1.csv')
item_properties_1_df.head()

<IPython.core.display.Javascript object>

,timestamp,itemid,property,value
0,1435460400000,460429,categoryid,1338
1,1441508400000,206783,888,1116713 960601 n277.200
2,1439089200000,395014,400,n552.000 639502 n720.000 424566
3,1431226800000,59481,790,n15360.000
4,1431831600000,156781,917,828513


In [11]:
item_properties_1_df.shape

(10999999, 4)

In [12]:
item_properties_2_df = pd.read_csv('TMP/item_properties_part2.csv')
item_properties_2_df.head()

<IPython.core.display.Javascript object>

,timestamp,itemid,property,value
0,1433041200000,183478,561,769062
1,1439694000000,132256,976,n26.400 1135780
2,1435460400000,420307,921,1149317 1257525
3,1431831600000,403324,917,1204143
4,1435460400000,230701,521,769062


In [13]:
item_properties_2_df.shape

(9275903, 4)

### Combine *item_properties_1* and *item_properties_2* together

In [14]:
item_properties = pd.concat([item_properties_1_df, item_properties_2_df], ignore_index = True)

<IPython.core.display.Javascript object>

In [15]:
item_properties.shape

(20275902, 4)

In [16]:
item_properties.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20275902 entries, 0 to 20275901
Data columns (total 4 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   timestamp  int64 
 1   itemid     int64 
 2   property   object
 3   value      object
dtypes: int64(2), object(2)
memory usage: 618.8+ MB


In [17]:
item_properties.head()

,timestamp,itemid,property,value
0,1435460400000,460429,categoryid,1338
1,1441508400000,206783,888,1116713 960601 n277.200
2,1439089200000,395014,400,n552.000 639502 n720.000 424566
3,1431226800000,59481,790,n15360.000
4,1431831600000,156781,917,828513


In [18]:
item_properties.isnull().sum()

timestamp    0
itemid       0
property     0
value        0
dtype: int64

### Get category IDs from item properties

In [19]:
category_properties = item_properties[item_properties['property'] == 'categoryid']
category_properties = category_properties[['itemid', 'value']].rename(columns = {'value': 'categoryid'})
category_properties.head()

,itemid,categoryid
0,460429,1338
140,281245,1277
151,35575,1059
189,8313,1147
197,55102,47


In [20]:
category_properties.shape

(788214, 2)

### Merge category information with events

In [21]:
events_data = events_df.merge(category_properties, on = 'itemid', how = 'left')
events_data.head()

,timestamp,visitorid,event,itemid,transactionid,categoryid
0,1433221332117,257597,view,355908,NaN,1173
1,1433224214164,992329,view,248676,NaN,1231
2,1433221999827,111016,view,318965,NaN,NaN
3,1433221955914,483717,view,253185,NaN,914
4,1433221337106,951259,view,367447,NaN,1613


In [22]:
events_data.shape

(5900263, 6)

In [23]:
events_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5900263 entries, 0 to 5900262
Data columns (total 6 columns):
 #   Column         Dtype  
---  ------         -----  
 0   timestamp      int64  
 1   visitorid      int64  
 2   event          object 
 3   itemid         int64  
 4   transactionid  float64
 5   categoryid     object 
dtypes: float64(1), int64(3), object(2)
memory usage: 270.1+ MB


In [24]:
events_data.isnull().sum()

timestamp              0
visitorid              0
event                  0
itemid                 0
transactionid    5856136
categoryid        255585
dtype: int64

### Combine *events data* and *category tree data* together

In [25]:
# Convert categoryid in the category tree dataset to object data type
category_df['categoryid'] = category_df['categoryid'].astype(str)

In [26]:
events_data = events_data.merge(category_df, on = 'categoryid', how = 'left')
events_data.head()

,timestamp,visitorid,event,itemid,transactionid,categoryid,parentid
0,1433221332117,257597,view,355908,NaN,1173,805.0
1,1433224214164,992329,view,248676,NaN,1231,901.0
2,1433221999827,111016,view,318965,NaN,NaN,NaN
3,1433221955914,483717,view,253185,NaN,914,226.0
4,1433221337106,951259,view,367447,NaN,1613,250.0


In [27]:
events_data.shape

(5900263, 7)

In [28]:
events_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5900263 entries, 0 to 5900262
Data columns (total 7 columns):
 #   Column         Dtype  
---  ------         -----  
 0   timestamp      int64  
 1   visitorid      int64  
 2   event          object 
 3   itemid         int64  
 4   transactionid  float64
 5   categoryid     object 
 6   parentid       float64
dtypes: float64(2), int64(3), object(2)
memory usage: 315.1+ MB


In [29]:
events_data.isnull().sum()

timestamp              0
visitorid              0
event                  0
itemid                 0
transactionid    5856136
categoryid        255585
parentid          255822
dtype: int64

### Convert the Unix timestamp to a datetime

In [30]:
events_data['datetime'] = pd.to_datetime(events_data['timestamp'])
events_data.head()

<IPython.core.display.Javascript object>

,timestamp,visitorid,event,itemid,transactionid,categoryid,parentid,datetime
0,1433221332117,257597,view,355908,NaN,1173,805.0,1970-01-01 00:23:53.221332117
1,1433224214164,992329,view,248676,NaN,1231,901.0,1970-01-01 00:23:53.224214164
2,1433221999827,111016,view,318965,NaN,NaN,NaN,1970-01-01 00:23:53.221999827
3,1433221955914,483717,view,253185,NaN,914,226.0,1970-01-01 00:23:53.221955914
4,1433221337106,951259,view,367447,NaN,1613,250.0,1970-01-01 00:23:53.221337106


In [31]:
# Clean the datetime column to remove the micro and nanoseconds
events_data['datetime'] = events_data['datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')
events_data.head()

,timestamp,visitorid,event,itemid,transactionid,categoryid,parentid,datetime
0,1433221332117,257597,view,355908,NaN,1173,805.0,1970-01-01 00:23:53
1,1433224214164,992329,view,248676,NaN,1231,901.0,1970-01-01 00:23:53
2,1433221999827,111016,view,318965,NaN,NaN,NaN,1970-01-01 00:23:53
3,1433221955914,483717,view,253185,NaN,914,226.0,1970-01-01 00:23:53
4,1433221337106,951259,view,367447,NaN,1613,250.0,1970-01-01 00:23:53
